<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/Final_package/drug_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 3: Disease Prediction and Drug recommendation**

In [0]:
import pandas as pd 
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

In [97]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Disease Prediction**

In [0]:
# Reading the symptoms dataset
sym_data = pd.read_csv("/content/drive/My Drive/gtdt/symptoms_data.csv")

In [99]:
sym_data.head()

,Disease,Heberden's node,Murphy's sign,Stahli's line,abdomen acute,abdominal bloating,abdominal tenderness,abnormal sensation,abnormally hard consistency,abortion,abscess bacterial,absences finding,achalasia,ache,adverse effect,adverse reaction,agitation,air fluid level,alcohol binge episode,alcoholic withdrawal symptoms,ambidexterity,angina pectoris,anorexia,anosmia,aphagia,apyrexial,arthralgia,ascites,asterixis,asthenia,asymptomatic,ataxia,atypia,aura,awakening early,barking cough,bedridden,behavior hyperactive,behavior showing increased motor activity,blackout,...,tenesmus,terrify,thicken,throat sore,throbbing sensation quality,tinnitus,tired,titubation,todd paralysis,tonic seizures,transaminitis,transsexual,tremor,tremor resting,tumor cell invasion,unable to concentrate,unconscious state,uncoordination,underweight,unhappy,unresponsiveness,unsteady gait,unwell,urge incontinence,urgency ofÂ micturition,urinary hesitation,urinoma,verbal auditory hallucinations,verbally abusive behavior,vertigo,vision blurred,vomiting,weepiness,weight gain,welt,wheelchair bound,wheezing,withdraw,worry,yellow sputum
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
sym_data = pd.concat([sym_data]*2, ignore_index=True)
cols = sym_data.columns
cols = cols[1:]

In [0]:
x = sym_data[cols]
y = sym_data['Disease']

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

## Training the data

Using decision tree and MNB classifier as it got highest accuracy

In [0]:
features = cols
feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

In [104]:
print ("DecisionTree")
dt = DecisionTreeClassifier(max_depth=120)
clf_dt=dt.fit(x_train,y_train)
print ("Acurracy: ", clf_dt.score(x_test,y_test))

DecisionTree
Acurracy:  0.8864144803742119


In [105]:
mnb = MultinomialNB()
clf_mnb=mnb.fit(x_train,y_train)
print ("Acurracy: ", clf_mnb.score(x_test,y_test))

Acurracy:  0.8774150905023388


In [106]:
input_features = [feature_dict['fever'], feature_dict['cough'], feature_dict['drool']]
input_features

[122, 70, 89]

In [0]:
arr =[]
for i in range(len(features)):
  if i == input_features[0]:
    i = int(i/input_features[0])
  elif i == input_features[1]:
    i = int(i/input_features[1])
  elif i == input_features[2]:
    i = int(i/input_features[2])
  else:
    i = 0
  arr.append(i)

In [0]:
arr = np.array(arr).reshape(-1,len(arr))

**Predicted Disease**

In [109]:
predicted_disease = (mnb.predict(arr))
print("The disease predicted based on given symptoms is : " + predicted_disease[0])

The disease predicted based on given symptoms is : Alzheimer's disease


Importing the preprocessed and merged dataset containing the drug names for every disease, the reviews, symptoms, rating and useful count of all the drugs.



# **Sentiment Analysis**

In [110]:
from google.colab import drive
drive.mount("/content/drive")


merged_data = pd.read_csv("/content/drive/My Drive/gtdt/Merged_Dataset.csv")
merged_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Drug,Disease,Review,Rating,UsefulCount,Symptoms
0,Aspirin,transient ischemic attack,"""No side effects, easy to take, no more sympt...",10,10,"['speech slurred', 'dysarthria', 'facial pares..."
1,Clopidogrel,transient ischemic attack,"""I&#039;ve been taking this medicine for a lit...",10,8,"['speech slurred', 'dysarthria', 'facial pares..."
2,Clopidogrel,transient ischemic attack,"""I took ibuprofen (2 caps at night for severe ...",6,13,"['speech slurred', 'dysarthria', 'facial pares..."
3,Clopidogrel,transient ischemic attack,"""After my VAD Stroke I am on plavix. I have a...",5,9,"['speech slurred', 'dysarthria', 'facial pares..."
4,Bayer Children's Aspirin,transient ischemic attack,"""No side effects, easy to take, no more sympt...",10,10,"['speech slurred', 'dysarthria', 'facial pares..."


**Performing sentiment analysis of drug review using VADER.**

In [111]:
pip install vaderSentiment

In [0]:
#Importing and installing the necessary library for VADER.

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [0]:
#Making a list of reviews to give it as input for vader analysis.
review_list = list(merged_data.Review)

#Initializing the necessary files.
sentiments = []
positive = []
negative = []
neutral = []
compound = []

In [0]:
for text in review_list:

  #Performing Vader Analysis on each review.
  com = analyser.polarity_scores(text)["compound"]
  pos = analyser.polarity_scores(text)["pos"]
  neu = analyser.polarity_scores(text)["neu"]
  neg = analyser.polarity_scores(text)["neg"]

  #Adding each value to the corresponding array
  positive.append(pos)
  negative.append(neg)
  neutral.append(neu)
  compound.append(com)
  sentiments.append({"Review":text,
                     "Positive": pos,
                     "Negative": neu,
                     "Neutral": neg,
                     "Compound": com})

sentiments_data = pd.DataFrame.from_dict(sentiments)

In [115]:
#Visualizing data from vader analysis.
sentiments_data

,Review,Positive,Negative,Neutral,Compound
0,"""No side effects, easy to take, no more sympt...",0.206,0.638,0.156,0.1779
1,"""I&#039;ve been taking this medicine for a lit...",0.098,0.762,0.140,-0.4092
2,"""I took ibuprofen (2 caps at night for severe ...",0.000,0.721,0.279,-0.8176
3,"""After my VAD Stroke I am on plavix. I have a...",0.055,0.945,0.000,0.6757
4,"""No side effects, easy to take, no more sympt...",0.206,0.638,0.156,0.1779
...,...,...,...,...,...
10055,"""Reminds me of Adderall but only a natural way...",0.081,0.919,0.000,0.5023
10056,"""My husband has Vascular Dementia for appox. 4...",0.177,0.674,0.149,0.3272
10057,"""I&#039;ve only been using this medication (pa...",0.164,0.638,0.198,-0.1531
10058,"""Makes me feel ill.""",0.000,0.517,0.483,-0.4215


In [0]:
#Adding the sentiment analysis columns to the merged dataset.
merged_data["Positive"] = positive
merged_data["Negative"] = negative
merged_data["Neutral"] = neutral
merged_data["Compound"] = compound
merged_data["Review_Sentiment"] = ''

In [117]:
#Visualizing the merged dataset with the sentiment analysis results.
merged_data.head()

,Drug,Disease,Review,Rating,UsefulCount,Symptoms,Positive,Negative,Neutral,Compound,Review_Sentiment
0,Aspirin,transient ischemic attack,"""No side effects, easy to take, no more sympt...",10,10,"['speech slurred', 'dysarthria', 'facial pares...",0.206,0.156,0.638,0.1779,
1,Clopidogrel,transient ischemic attack,"""I&#039;ve been taking this medicine for a lit...",10,8,"['speech slurred', 'dysarthria', 'facial pares...",0.098,0.140,0.762,-0.4092,
2,Clopidogrel,transient ischemic attack,"""I took ibuprofen (2 caps at night for severe ...",6,13,"['speech slurred', 'dysarthria', 'facial pares...",0.000,0.279,0.721,-0.8176,
3,Clopidogrel,transient ischemic attack,"""After my VAD Stroke I am on plavix. I have a...",5,9,"['speech slurred', 'dysarthria', 'facial pares...",0.055,0.000,0.945,0.6757,
4,Bayer Children's Aspirin,transient ischemic attack,"""No side effects, easy to take, no more sympt...",10,10,"['speech slurred', 'dysarthria', 'facial pares...",0.206,0.156,0.638,0.1779,


Based on the compound value we can determine whether the overall sentiment of the review is postive, negative or neutral. Below will be the threshold we shall be using for classifying the review sentiment class:

Positive sentiment: compound score >= 0.05

Neutral sentiment : -0.05 < compound score < 0.05

Negative sentiment: compound score <= *-0.05*

In [0]:
merged_data.loc[merged_data['Compound'] >= 0.05, 'Review_Sentiment'] = 'Positive'
merged_data.loc[merged_data['Compound'] <= -0.05, 'Review_Sentiment'] = 'Negative'
merged_data["Review_Sentiment"].replace('', 'Neutral', inplace = True)

In [0]:
#Dropping the columns Positive, Negative, Neutral and Compound.
merged_data = merged_data.drop(columns = ["Positive", "Negative", "Neutral", "Compound"])

In [0]:
# Rearranging the columns in different order.
merged_data = merged_data[["Disease", "Drug", "Symptoms", "Review", "Review_Sentiment", "Rating", "UsefulCount"]]

In [121]:
#Visualizing the merged dataset after adding the sentiment analysis results.
merged_data.head()

,Disease,Drug,Symptoms,Review,Review_Sentiment,Rating,UsefulCount
0,transient ischemic attack,Aspirin,"['speech slurred', 'dysarthria', 'facial pares...","""No side effects, easy to take, no more sympt...",Positive,10,10
1,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""I&#039;ve been taking this medicine for a lit...",Negative,10,8
2,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""I took ibuprofen (2 caps at night for severe ...",Negative,6,13
3,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""After my VAD Stroke I am on plavix. I have a...",Positive,5,9
4,transient ischemic attack,Bayer Children's Aspirin,"['speech slurred', 'dysarthria', 'facial pares...","""No side effects, easy to take, no more sympt...",Positive,10,10


In [0]:
#Exporting this merged dataset as csv file.
merged_data.to_csv(r'drive/My Drive/gtdt/Sentiment_analysis.csv', index=False)

## **Weighted average of rating and useful count**

In [0]:
#Reading the dataset with sentiment analysis of reviews
data = pd.read_csv('drive/My Drive/gtdt/Sentiment_analysis.csv')

In [124]:
data.head()

,Disease,Drug,Symptoms,Review,Review_Sentiment,Rating,UsefulCount
0,transient ischemic attack,Aspirin,"['speech slurred', 'dysarthria', 'facial pares...","""No side effects, easy to take, no more sympt...",Positive,10,10
1,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""I&#039;ve been taking this medicine for a lit...",Negative,10,8
2,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""I took ibuprofen (2 caps at night for severe ...",Negative,6,13
3,transient ischemic attack,Clopidogrel,"['speech slurred', 'dysarthria', 'facial pares...","""After my VAD Stroke I am on plavix. I have a...",Positive,5,9
4,transient ischemic attack,Bayer Children's Aspirin,"['speech slurred', 'dysarthria', 'facial pares...","""No side effects, easy to take, no more sympt...",Positive,10,10


In [0]:
#Sorting the data based on drug name
data = data.sort_values(['Drug'])

In [0]:
# Creating a function to calculate weighted average
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [127]:
data.groupby(["Disease", "Drug"]).apply(wavg, "Rating", "UsefulCount")

Disease                    Drug                    
alzheimer's disease        Aricept                      5.807916
                           Aricept ODT                  9.000000
                           Donepezil                    5.503089
                           Exelon                       7.526027
                           Galantamine                  8.510638
                                                         ...    
schizophrenia              Zyprexa Zydis                6.304348
transient ischemic attack  Aspirin                     10.000000
                           Bayer Children's Aspirin    10.000000
                           Clopidogrel                  6.766667
                           Plavix                       5.000000
Length: 403, dtype: float64

In [0]:
# Creating a dataframe of Drug and its average rating
data_wavg = pd.DataFrame(data.groupby(["Drug"]).apply(wavg, "Rating", "UsefulCount").reset_index())

In [0]:
data_wavg = data_wavg.rename(columns={0: "Rating_Wavg"})

In [130]:
data_wavg.head()

,Drug,Rating_Wavg
0,Abilify,7.350122
1,Abilify Maintena,8.714286
2,Acetaminophen / chlorpheniramine,8.000000
3,Acetaminophen / phenyltoloxamine,10.000000
4,Acetazolamide,6.376068


In [0]:
merged_wavg = pd.merge(data_wavg, data, on='Drug')

In [0]:
# Merging the weighted average column with the dataset
merged_wavg.drop(columns=['Symptoms','Rating'], inplace=True)
merged_wavg = merged_wavg[['Disease','Drug', 'Review', 'Review_Sentiment', 'Rating_Wavg', 'UsefulCount']]

In [133]:
merged_wavg.head()

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,schizophrenia,Abilify,"""This medication has helped my son a great dea...",Positive,7.350122,40
1,schizophrenia,Abilify,"""This medication made me feel like my brain wa...",Negative,7.350122,10
2,schizophrenia,Abilify,"""I&#039;ve used 5mg daily for three weeks. My ...",Positive,7.350122,30
3,schizophrenia,Abilify,"""Excellent medication! When I was first on Abi...",Negative,7.350122,11
4,schizophrenia,Abilify,"""I used to hear bad voices in my head,and thou...",Negative,7.350122,107


In [134]:
merged_wavg['Disease'].nunique()

26

# **Drug Recommendation**

In [135]:
#Sorting dataset and grouping by disease
merged_wavg = merged_wavg.sort_values(['Disease','Rating_Wavg'],ascending=False, ignore_index=True).groupby('Disease').head(10060)
merged_wavg

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,transient ischemic attack,Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
1,transient ischemic attack,Bayer Children's Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
2,transient ischemic attack,Clopidogrel,"""I&#039;ve been taking this medicine for a lit...",Negative,6.766667,8
3,transient ischemic attack,Clopidogrel,"""I took ibuprofen (2 caps at night for severe ...",Negative,6.766667,13
4,transient ischemic attack,Clopidogrel,"""After my VAD Stroke I am on plavix. I have a...",Positive,6.766667,9
...,...,...,...,...,...,...
10055,alzheimer's disease,Namenda,"""I have taken it for about 4-5 yes. I feel ab...",Positive,4.100946,38
10056,alzheimer's disease,Namenda,"""My mother has been taking Namenda for Alzheim...",Negative,4.100946,120
10057,alzheimer's disease,Namenda,"""My mother has been treated with the Exelon (p...",Negative,4.100946,109
10058,alzheimer's disease,Namenda,"""My wife (74) has been taking Namenda since th...",Positive,4.100946,55


In [136]:
# Exporting the dataset with weighted average
merged_wavg = merged_wavg.to_csv("/content/drive/My Drive/gtdt/Drug_Recommender.csv", index=False)
merged_wavg = pd.read_csv("/content/drive/My Drive/gtdt/Drug_Recommender.csv")
merged_wavg

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,transient ischemic attack,Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
1,transient ischemic attack,Bayer Children's Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
2,transient ischemic attack,Clopidogrel,"""I&#039;ve been taking this medicine for a lit...",Negative,6.766667,8
3,transient ischemic attack,Clopidogrel,"""I took ibuprofen (2 caps at night for severe ...",Negative,6.766667,13
4,transient ischemic attack,Clopidogrel,"""After my VAD Stroke I am on plavix. I have a...",Positive,6.766667,9
...,...,...,...,...,...,...
10055,alzheimer's disease,Namenda,"""I have taken it for about 4-5 yes. I feel ab...",Positive,4.100946,38
10056,alzheimer's disease,Namenda,"""My mother has been taking Namenda for Alzheim...",Negative,4.100946,120
10057,alzheimer's disease,Namenda,"""My mother has been treated with the Exelon (p...",Negative,4.100946,109
10058,alzheimer's disease,Namenda,"""My wife (74) has been taking Namenda since th...",Positive,4.100946,55


In [0]:
# Taking drugs only with positive reviews for recommendation
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Negative"].index)
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Neutral"].index)

In [138]:
merged_wavg.head()

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,transient ischemic attack,Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
1,transient ischemic attack,Bayer Children's Aspirin,"""No side effects, easy to take, no more sympt...",Positive,10.000000,10
4,transient ischemic attack,Clopidogrel,"""After my VAD Stroke I am on plavix. I have a...",Positive,6.766667,9
5,transient ischemic attack,Plavix,"""After my VAD Stroke I am on plavix. I have a...",Positive,5.000000,9
6,schizophrenia,Mellaril,"""Moochie used to take Melleril, was always cal...",Positive,10.000000,22


In [0]:
# Taking predicted disease as input and recommending drug based on highest weighted average and useful count of ratings
groupedByCount = merged_wavg.groupby(['Disease', 'Drug', 'Rating_Wavg'])['UsefulCount'].sum().reset_index()

In [140]:
groupedByCount

,Disease,Drug,Rating_Wavg,UsefulCount
0,alzheimer's disease,Aricept,5.807916,1454
1,alzheimer's disease,Donepezil,5.503089,1559
2,alzheimer's disease,Exelon,7.526027,80
3,alzheimer's disease,Galantamine,8.510638,71
4,alzheimer's disease,Memantine,4.286241,218
...,...,...,...,...
327,schizophrenia,Zyprexa Zydis,6.304348,19
328,transient ischemic attack,Aspirin,10.000000,10
329,transient ischemic attack,Bayer Children's Aspirin,10.000000,10
330,transient ischemic attack,Clopidogrel,6.766667,9


In [0]:
groupedByDisease = groupedByCount.groupby('Disease')

Recommending drug based on the disease predicted above

In [144]:
recommended_drug = pd.DataFrame(groupedByDisease.get_group((predicted_disease[0]).lower()).nlargest(3, ['Rating_Wavg', 'UsefulCount']))
recommended_drug

,Disease,Drug,Rating_Wavg,UsefulCount
7,alzheimer's disease,Razadyne ER,10.000000,19
3,alzheimer's disease,Galantamine,8.510638,71
2,alzheimer's disease,Exelon,7.526027,80


In [146]:
print("Recommended drugs for this disease are:\n ", recommended_drug["Drug"].unique())

Recommended drugs for this disease are:
  ['Razadyne ER' 'Galantamine' 'Exelon']


### **Side effects for the recommended drug**